**Objectives:**
- recommend the next song for an user base on his current playlist

**Assumptions:**
- tracks of a given playlist are the favorite songs of the user

**Methodology:**
1. Collaborative filtering
2. Initial sample for each user will take 70% of the tracks in the playlist (check if the sorting of the musics in the playlist has any information - date of inclusion, user sorting, etc) and then use the remaining 30% of the tracks as test.
3. **If we have time:** Hybrid model by adding *Content based filtering* for playlists with few tracks
    - Track features: genre, time, top_track, etc.
**Concerns:**
- cold start: playlists with few tracks
- rare tracks: treatment to be applied to the tracks with low frequency on the database
- only positive rating: we will have only tracks that the user liked, we do not have any data about tracks that they did not like.


**Python Libraries:**
- Sagemaker
- Elasticsearch
- PySpark

https://developer.spotify.com/documentation/web-api/
https://developer.spotify.com/
https://developer.spotify.com/dashboard/login

In [1]:
# Default libraries: pandas, numpy
# import beakerx
import cProfile
import datetime
import feather
import math
import os
import pickle
import sys
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
# import urbangrammar_graphics as ugg

from IPython.display import Markdown as md
from collections import Counter
from clustergram import Clustergram
from datetime import datetime
from itertools import groupby

from sklearn import __version__ as sklearn_version
from sklearn.decomposition import PCA
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, AdaBoostClassifier, BaggingClassifier
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score, classification_report
from sklearn.model_selection import train_test_split, cross_validate, learning_curve, GridSearchCV, RandomizedSearchCV, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import scale, StandardScaler, MinMaxScaler, RobustScaler
from sklearn.tree import DecisionTreeClassifier

from xgboost import XGBClassifier

# from library.sb_utils import save_file

pd.options.display.float_format = "{:.2f}".format
# plt.rcParams.update({'font.size': 18})

In [2]:
from dotenv import load_dotenv

load_dotenv()
SPOTIPY_API = os.getenv('SPOTIPY_API')
SPOTIPY_API_KEY = os.getenv('SPOTIPY_API_KEY')
scope=''

# https://spotipy.readthedocs.io/en/latest/#
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id=SPOTIPY_API, client_secret=SPOTIPY_API_KEY)

# help(sp)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
def get_playlist_detail(user, playlist_id):
    artists = []
    albums =[]
    tracks = []
    popularity = []
    artists_id = []
    albums_id =[]
    tracks_id = []
    
    playlist = sp.user_playlist(user, playlist_id)
    
    
    for item in playlist['tracks']['items']:
#         print(item)
        for artist in item['track']['album']['artists']:
            artists.append(artist['name'])
            artists_id.append(artist['id'])
        albums.append(item['track']['album']['name'])
        tracks.append(item['track']['name'])
        popularity.append(item['track']['popularity'])
        
        albums_id.append(item['track']['album']['id'])
        tracks_id.append(item['track']['id'])
        
    return playlist_id, artists, albums, tracks, popularity, artists_id, albums_id, tracks_id;

# playlist_id, artists, albums, tracks, popularity, artists_id, albums_id, tracks_id = get_playlist_detail('aweben', '0ZX4VUgxREEHGK5NPTeCX3')

In [4]:
n = 100
for i in range(0,n,1):
    playlist = sp.search(q='year:2020', type='playlist', limit=1,offset=i,market='US')
    
    playlist_id, artists, albums, tracks, popularity, artists_id, albums_id, tracks_id = get_playlist_detail(playlist['playlists']['owner'],playlist['playlists']['id'])

In [5]:
# https://towardsdatascience.com/stop-using-csvs-for-storage-this-file-format-is-150-times-faster-158bd322074e
# help(feather)
# df = pd.DataFrame()

# df.to_feather('1M.feather')
# feather.write_dataframe(df, 'feather_test.feather')

In [6]:
type(playlist)


dict

In [7]:
playlist

{'playlists': {'href': 'https://api.spotify.com/v1/search?query=year%3A2020&type=playlist&market=US&offset=99&limit=1',
  'items': [{'collaborative': False,
    'description': 'By popular demand...heres 100 songs that helped get me through 2020. For the full selection listen to the End of Year Mixtapes part one &amp; two on BBC Sounds. Available for another few weeks. Tune in to the radio show on BBC Radio 1 every wednesday night for brand new music weekly. Love to one and all',
    'external_urls': {'spotify': 'https://open.spotify.com/playlist/3js9I39AXh0FB7U0rDRfHF'},
    'href': 'https://api.spotify.com/v1/playlists/3js9I39AXh0FB7U0rDRfHF',
    'id': '3js9I39AXh0FB7U0rDRfHF',
    'images': [{'height': None,
      'url': 'https://i.scdn.co/image/ab67706c0000bebb9958788ecdb3bb2426d6f75c',
      'width': None}],
    'name': 'Benji B. - 2020 End of Year Mixtape',
    'owner': {'display_name': 'Benji B',
     'external_urls': {'spotify': 'https://open.spotify.com/user/benjib'},
     'hr

In [11]:
playlist['items']['playlists']['owner']

KeyError: 'items'